In [1]:
!pip install boto3

In [2]:
!pip install psycopg2-binary

In [3]:
import sys
sys.path.append('./../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.features.build_features import clean
from src.models.train_model import get_data, evaluate


from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id, countDistinct, approxCountDistinct, when

from pyspark.ml.feature import OneHotEncoder, StringIndexer, Imputer, VectorAssembler, StandardScaler, PCA
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

from collections import defaultdict

import pandas as pd



Nueva cubeta


## Pruebas

In [11]:
import sys
sys.path.append('./../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2


from src.utils.db_utils import execute_sql
from src.utils.s3_utils import create_bucket
from src.models.train_model import run_model,prepare_data
from src.models.save_model import parse_filename

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import sys
sys.path.append('./../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.models.train_model import get_data, run_model,prepare_data, get_models_params_dic,evaluate,add_meta_data
from src.models.save_model import save_upload, parse_filename

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
numIt = 2
numPCA = 1
bucname = "model-dpa"

In [15]:
objetivo = "cancelled"
model_name = "LR"
hyperparams = {"iter": int(numIt),
                "pca": int(numPCA)}

output_path = parse_filename(objetivo, model_name, hyperparams)
output_path = "s3://" + str(bucname) +  output_path[1:] + ".model.zip"

output_path 

's3://model-dpa/24042020_cancelled_LR_=#iter#-%2$%#pca#-%1&.zip'

In [22]:
objetivo = "cancelled"
model = "LR"
hyperparams = {"iter": int(numIt),
                "pca": int(numPCA)}

# Esta funcion ya guarda los metadatos en un BD
# Corre los modelos y el mejor lo guarda en un S3 y guarda toda la info
# Toma la base de semantic
run_model(objetivo, model, hyperparams, True)

Selecting rows from table using cursor.fetchall
2
Print each row and it's columns values
('24042020', 'cancelled', 'LR', '{"iter": 2, "pca": 1}', '0.623870', '0.017011', '0.9858980702622464', '0.9858980702622464', '0.9858980702622464', '42.935035705566406', '0.2', '15958')
('24042020', 'cancelled', 'LR', '{"iter": 2, "pca": 1}', '0.623870', '0.017011', '0.9858980702622464', '0.9858980702622464', '0.9858980702622464', '42.84245729446411', '0.2', '15958')
PostgreSQL connection is closed
Nueva cubeta
models-dpa
test-aws-boto
/home/jovyan/work/src/models
[StringIndexer_a2afb507a0e7, StringIndexer_d766eccd63d6, StringIndexer_4518684b31c1, StringIndexer_f498d20d1b6a, StringIndexer_3c492e302635, StringIndexer_30bb471521ab, StringIndexer_30e69d4c8cbe, StringIndexer_17b41f0e7983, OneHotEncoder_495426ba9b67, OneHotEncoder_5be587f4f4b3, OneHotEncoder_bf0ef4ce96d9, OneHotEncoder_86511d556215, OneHotEncoder_d68109e588ef, OneHotEncoder_4029f6ad0024, OneHotEncoder_b80a6054cf47, OneHotEncoder_dfd8cd80

In [8]:
from pyspark.sql.functions import monotonically_increasing_id, countDistinct, approxCountDistinct, when
objetivo = "cancelled"
model_name = "LR"
hyperparams = {"pca": 1, "iter":1}
test_split = 0.2

df = get_data(False)
first_stages,df = prepare_data(df)

df = df.withColumn("label",  when(df.rangoatrasohoras == objetivo, 1.0).otherwise(0.0))

# Selecciona el modelo
model_dic, paramGrid_dic  = get_models_params_dic()
clr_model = model_dic[model_name]

# Parametros especificos
num_it = int(hyperparams["iter"])
if num_it > 0:
    clr_model.setMaxIter(num_it)

# Adds new stages
num_pca = int(hyperparams["pca"])
if num_pca > 0:
    stage_pca = PCA(k = num_pca,inputCol = "scaled_features", 
                        outputCol = "features")
else:
    stage_pca = PCA(k = 8,inputCol = "scaled_features", 
                outputCol = "features")

# Checar que no se haya corrido este modelo 

print("Modelo evaluado: ", clr_model, "con params: ", clr_model.explainParams())



/home/jovyan/work/src/models
[StringIndexer_5134f6b8fde8, StringIndexer_7a84310d5566, StringIndexer_f46240e9982f, StringIndexer_f37609338763, StringIndexer_967f904a98c9, StringIndexer_6341f550810f, StringIndexer_9e99f7b8b0e5, StringIndexer_c6d49137d73a, OneHotEncoder_ab77ed42b9fa, OneHotEncoder_0ff4fdc6e344, OneHotEncoder_332d66c2d4ea, OneHotEncoder_a5726697cd40, OneHotEncoder_f8e085c55a0c, OneHotEncoder_263e5498387d, OneHotEncoder_0109a878a11d, OneHotEncoder_17b86f63c8d9, Imputer_b00167a33c03, Imputer_422bdae3ee89, VectorAssembler_52c22faeb163, StandardScaler_89fbf24c9a80]


NameError: name 'PCA' is not defined

In [ ]:
first_stages 

In [3]:

# Creates Pipeline
pipeline = Pipeline(stages= first_stages + [stage_pca, clr_model])

df_train, df_test = df.randomSplit([(1-test_split),test_split ], 123)

start = time.time()
cvModel  = pipeline.fit(df_train)
end = time.time()

prediction = cvModel.transform(df_test)
log = evaluate(prediction)


#Guarda en s3
save_upload(cvModel, objetivo, model_name, hyperparams)

# Sube metadatos a RDS
# --- Metadata -----
train_time = end - start
train_nrows = df_train.count()
# -------------------
add_meta_data(objetivo, model_name,hyperparams, log,train_time, test_split, train_nrows)


NameError: name 'Pipeline' is not defined

In [2]:
def imputa_categoricos(df, ignore,data_types):
    strings_used = [var for var in data_types["StringType"] if var not in ignore]

    missing_data_fill = {}
    for var in strings_used:
        missing_data_fill[var] = "missing"

    df = df.fillna(missing_data_fill)
    return df

def ignore_list(df, data_types):
    counts_summary = df.agg(*[countDistinct(c).alias(c) for c in data_types["StringType"]])
    counts_summary = counts_summary.toPandas()

    counts = pd.Series(counts_summary.values.ravel())
    counts.index = counts_summary.columns

    sorted_vars = counts.sort_values(ascending = False)
    ignore = list((sorted_vars[sorted_vars >100]).index)
    return ignore

def get_data_types(df):
    data_types = defaultdict(list)
    for entry in df.schema.fields:
        data_types[str(entry.dataType)].append(entry.name)
    return data_types



In [3]:
def create_pipeline(df, ignore):
    # Esto lo ponemos aqui para poder modificar las 
    #variables de los estimadores/transformadores
    data_types = get_data_types(df)    
    #--------------------------------------
    
    # -------------- STRING --------------
    strings_used = [var for var in data_types["StringType"] if var not in ignore]

    # -------------- DOUBLE --------------
    numericals_double = [var for var in data_types["DoubleType"] if var not in ignore]
    numericals_double_imputed = [var + "_imputed" for var in numericals_double]

    # -------------- INTEGERS --------------
    from pyspark.sql.types import IntegerType, DoubleType
    numericals_int = [var for var in data_types["IntegerType"] if var not in ignore]
    
    for c in numericals_int:
        df = df.withColumn(c, df[c].cast(DoubleType()))
        df = df.withColumn(c, df[c].cast("double"))
        
    numericals_int_imputed = [var + "_imputed" for var in numericals_int]
    # =======================================

    ## %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ##            P I P E L I N E
    ## %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    # ============= ONE HOT ENCODING ================
    from pyspark.ml.feature import OneHotEncoder, StringIndexer
    stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
    stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

    # =============== IMPUTADORES ====================
    from pyspark.ml.feature import Imputer
    stage_imputer_double = Imputer(inputCols = numericals_double, 
                                   outputCols = numericals_double_imputed) 
    stage_imputer_int = Imputer(inputCols = numericals_int, 
                                outputCols = numericals_int_imputed) 

    # ============= VECTOR ASESEMBLER ================
    from pyspark.ml.feature import VectorAssembler

    features =  numericals_double_imputed \
              + [var + "_one_hot" for var in strings_used]
    stage_assembler = VectorAssembler(inputCols = features, outputCol= "assem_features")

    # ==================== SCALER =======================
    from pyspark.ml.feature import StandardScaler
    stage_scaler = StandardScaler(inputCol= stage_assembler.getOutputCol(), 
                                  outputCol="scaled_features", withStd=True, withMean=True)

    # ================== PIPELINE ===================
    stages= stage_string + stage_one_hot +  [             # Categorical Data
                               stage_imputer_double,
                               stage_imputer_int,        # Data Imputation
                               stage_assembler,          # Assembling data
                               stage_scaler] 
                          
    ## Tenemos que regesar el df porque las variables int las combierte en double
    return  stages , df

In [4]:
def get_models_params_dic():
    stage_pca = PCA(k = 15,inputCol = "scaled_features",
                        outputCol = "features")


    lr = LogisticRegression()

    lr_paramGrid = ParamGridBuilder() \
    .addGrid(stage_pca.k, [1]) \
    .addGrid(lr.maxIter, [1]) \
    .build()

    dt = DecisionTreeClassifier()

    dt_paramGrid = ParamGridBuilder() \
    .addGrid(stage_pca.k, [1]) \
    .addGrid(dt.maxDepth, [2]) \
    .build()

    paramGrid_dic= {"LR":lr_paramGrid,"DT":dt_paramGrid}
    model_dic = {"LR":lr,"DT":dt}

    return model_dic,paramGrid_dic

In [5]:
def prepare_data(df):
    data_types = get_data_types(df)
    ignore =   ignore_list(df, data_types) 
    illegal = [s for s in df.columns if "del" in s]
    extra_illegal = ['cancelled', 'rangoatrasohoras']
    legal = [var for var in df.columns if (var not in ignore and var not in illegal and var not in extra_illegal)]
    lista_objetivos = df.select('rangoatrasohoras').distinct().rdd.map(lambda r: r[0]).collect()

    df = imputa_categoricos(df, ignore,data_types)
    
    df_legal = df[legal]
    y = df[['rangoatrasohoras']]
    
    df_legal = df_legal.withColumn("id", monotonically_increasing_id())
    y = y.withColumn("id", monotonically_increasing_id())
    
    stages, df_new = create_pipeline(df_legal, ignore)

    df_junto = df_new.join(y, "id", "outer").drop("id")

    return df_junto, stages
    

    
    
def run_model(objetivo, model_name, hyperparams, luigi= False):
    df = get_data(luigi)
    df, first_stages = prepare_data(df)
  
    df = df.withColumn("label",  when(df.rangoatrasohoras == objetivo, 1.0).otherwise(0.0))
    
    # Selecciona el modelo
    model_dic, paramGrid_dic  = get_models_params_dic()
    clr_model = model_dic[model_name]
    
    # Parametros especificos
    num_it = int(hyperparams["iter"])
    if num_it > 0:
        clr_model.setMaxIter(num_it)
    
    # Adds new stages
    num_pca = int(hyperparams["pca"])
    if num_pca > 0:
        stage_pca = PCA(k = num_pca,inputCol = "scaled_features", 
                            outputCol = "features")
    else:
        stage_pca = PCA(k = 8,inputCol = "scaled_features", 
                    outputCol = "features")
    
    # Checar que no se haya corrido este modelo 
    
    print("Modelo evaluado: ", clr_model, "con params: ", clr_model.explainParams())
    
    # Creates Pipeline
    pipeline = Pipeline(stages= first_stages + [stage_pca, clr_model])

    df_train, df_test = df.randomSplit([0.8,0.2], 123)

    cvModel  = pipeline.fit(df_train)
    prediction = cvModel.transform(df_test)
    evaluate(prediction)
    
    #Sacar metadatos
    #Insertar metadatos
    #Guardar modelos
    

In [39]:
objetivo = "cancelled"
model = "LR"
hyperparams = {"pca": 1, "iter":1}

run_model(objetivo, model, hyperparams)

/home/jovyan/work/src/models
Modelo evaluado:  LogisticRegression_da34f1229d49 con params:  aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on interc

In [ ]:

import time


objetivo = "cancelled"
model_name = "LR"
hyperparams = {"pca": 1, "iter":1}
test_split = 0.2

def run_model(objetivo, model_name, hyperparams, luigi= False, test_split = 0.2):
    df = get_data(False)
    df, first_stages = prepare_data(df)

    df = df.withColumn("label",  when(df.rangoatrasohoras == objetivo, 1.0).otherwise(0.0))

    # Selecciona el modelo
    model_dic, paramGrid_dic  = get_models_params_dic()
    clr_model = model_dic[model_name]

    # Parametros especificos
    num_it = int(hyperparams["iter"])
    if num_it > 0:
        clr_model.setMaxIter(num_it)

    # Adds new stages
    num_pca = int(hyperparams["pca"])
    if num_pca > 0:
        stage_pca = PCA(k = num_pca,inputCol = "scaled_features", 
                            outputCol = "features")
    else:
        stage_pca = PCA(k = 8,inputCol = "scaled_features", 
                    outputCol = "features")

    # Checar que no se haya corrido este modelo 

    print("Modelo evaluado: ", clr_model, "con params: ", clr_model.explainParams())

    # Creates Pipeline
    pipeline = Pipeline(stages= first_stages + [stage_pca, clr_model])

    df_train, df_test = df.randomSplit([(1-test_split),test_split ], 123)

    start = time.time()
    cvModel  = pipeline.fit(df_train)
    end = time.time()

    prediction = cvModel.transform(df_test)
    log = evaluate(prediction)


    #Guarda en s3
    save_upload(cvModel, objetivo, model_name, hyperparams)

    # Sube metadatos a RDS
    # --- Metadata -----
    train_time = end - start
    train_nrows = df_train.count()
    # -------------------
    add_meta_data(objetivo, model_name,hyperparams, log,train_time, test_split, train_nrows)




In [86]:
AUROC = log['AUROC']
AUPR = log['AUPR']
precision = log['precision']
recall = log['recall']
f1 =log['F1 Measure'] 
train_nrows = df_train.count()

## Save model

In [57]:
from src.models.save_model import save_upload



Nueva cubeta


In [59]:
save_upload(cvModel, objetivo, model_name, hyperparams)

22042020_cancelled_LR_=#pca#-%1$%#iter#-%1&.model.zip file is created successfully!


## Metadatos

In [67]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9 MB 1.5 MB/s eta 0:00:01


In [71]:
from src.utils.db_utils import execute_sql,insert_query,show_select
file_dir = "./../src/utils/sql/metada_model.sql"
# Crear schema
execute_sql(file_dir)



PostgreSQL connection is closed


In [75]:
from datetime import date, datetime

today = date.today()
d1 = today.strftime("%d%m%Y")
d1
datetime.now()

datetime.datetime(2020, 4, 22, 21, 28, 36, 73245)

In [90]:
from datetime import date
from src.utils.db_utils import execute_sql,insert_query,show_select

def add_meta_data(objetivo, model_name,hyperparams, log,train_time, test_split, train_nrows):
    AUROC = log['AUROC']
    AUPR = log['AUPR']
    precision = log['precision']
    recall = log['recall']
    f1 =log['F1 Measure'] 
    today = date.today()
    d1 = today.strftime("%d%m%Y")

    query = """ INSERT INTO metadatos.models (fecha, objetivo, model_name, hyperparams, AUROC, AUPR, precision, recall, f1, train_time, test_split, train_nrows ) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s  ) """
    values = (d1,
             objetivo, model_name,
             json.dumps(hyperparams),
             AUROC, AUPR, precision, recall, f1, train_time, test_split, train_nrows)
    insert_query(query, values)   

PostgreSQL connection is closed


In [91]:
show_select("select * from metadatos.models")

Selecting rows from table using cursor.fetchall
1
Print each row and it's columns values
('22042020', 'cancelled', 'LR', '{"pca": 1, "iter": 1}', '0.623870', '0.017011', '0.9858980702622464', '0.9858980702622464', '0.9858980702622464', '-48.46292996406555', '0.2', '15958')
PostgreSQL connection is closed


## Experimentos

In [19]:
!pip install boto3
from src.utils.s3_utils import describe_s3, get_s3_objects

     |████████████████████████████████| 128 kB 701 kB/s eta 0:00:01
     |████████████████████████████████| 69 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 74 kB/s  eta 0:00:01
     |████████████████████████████████| 547 kB 47 kB/s  eta 0:00:01
Nueva cubeta
models-dpa
models-dpa


In [53]:
trained_model = cvModel.stages[-1]

In [54]:
type(trained_model)

pyspark.ml.classification.LogisticRegressionModel

In [12]:
import json 

In [32]:
import json 
from datetime import date

def parse_filename(objetivo, model_name, hyperparams):    
    para_string = json.dumps(hyperparams) 
    para_string = para_string.replace(" ", "%")
    para_string = para_string.replace('"', "#")
    para_string = para_string.replace('}', "&")
    para_string = para_string.replace('{', "=")
    para_string = para_string.replace(':', "-")
    para_string = para_string.replace(',', "$")
    
    today = date.today()
    d1 = today.strftime("%d%m%Y")

    saved_model_name = "./" + d1 + "_" + objetivo + "_" + model_name + "_" + para_string 

    return saved_model_name


def save_upload(cvModel, objetivo, model_name, hyperparams,bucket_name = "models-dpa"):
    trained_model = cvModel.stages[-1]
    
    saved_model_name = parse_filename(objetivo, model_name, hyperparams) + ".model"
    key_name = saved_model_name[2:]
    
    # Save model
    trained_model.save(saved_model_name)

    # Zip model 
    zip_model(key_name)
    
    new_saved_model = saved_model_name +".zip"
    new_key_name = new_saved_model[2:] 

    # Upload file
    upload_file_to_bucket(new_saved_model, bucket_name, new_key_name)
        

In [33]:
save_and_zip(cvModel, objetivo, model_name, hyperparams)

In [34]:
saved_model_name = parse_filename(objetivo, model_name, hyperparams)
saved_model_name

'./22042020_cancelled_LR_=#pca#-%1$%#iter#-%1&.model'

## Upload to S3

In [21]:
import boto3
from src import (
    BUCKET,
    MY_REGION,
    MY_REGION2,
    MY_PROFILE,
    MY_KEY,
    MY_AMI ,
    MY_VPC ,
    MY_GATEWAY,
    MY_SUBNET,
    MY_GROUP
)


ses = boto3.session.Session(profile_name=MY_PROFILE, region_name=MY_REGION,)
s3 = ses.resource('s3')
bucket_name = BUCKET

In [54]:
bucket_name = "models-dpa"
#my_bucket = s3_resource.Bucket(bucket_name)

model_dir = parse_filename(objetivo, model_name, hyperparams)
key_name = saved_model_name[2:]

zip_model(key_name)

22042020_cancelled_LR_=#pca#-%1$%#iter#-%1&.model.zip file is created successfully!


In [53]:
import os
import zipfile
 
#Declare the function to return all file paths of the particular directory
def retrieve_file_paths(dirName):
    # setup file paths variable
    filePaths = []
    # Read all directory, subdirectories and file lists
    for root, directories, files in os.walk(dirName):
        for filename in files:
            # Create the full filepath by using os module.
            filePath = os.path.join(root, filename)
            filePaths.append(filePath)
    # return all paths
    return filePaths
  
def zip_model(dir_name):
    #https://linuxhint.com/python_zip_file_directory/
    # Call the function to retrieve all files and folders of the assigned directory
    filePaths = retrieve_file_paths(dir_name)
    # writing files to a zipfile
    zip_file = zipfile.ZipFile(dir_name+'.zip', 'w')
    with zip_file:
        # writing each file one by one
        for file in filePaths:
            zip_file.write(file)
    print(dir_name+'.zip file is created successfully!')

In [55]:
bucket_name = "models-dpa"
#my_bucket = s3_resource.Bucket(bucket_name)

model_dir = parse_filename(objetivo, model_name, hyperparams) +".zip"
key_name = saved_model_name[2:] 

ses = boto3.session.Session(profile_name=MY_PROFILE, region_name=MY_REGION,)
s3 = ses.resource('s3')

# Escribimos el archivo al bucket, usando el binario
s3.meta.client.upload_file(model_dir, bucket_name, key_name)

In [113]:
trained_model.save(saved_model_name)

In [114]:
from pyspark.ml.classification import RandomForestClassificationModel, LogisticRegressionModel
import os 

prueba = LogisticRegressionModel.load(saved_model_name)

In [115]:
prueba

LogisticRegressionModel: uid = LogisticRegression_2e27ecb108eb, numClasses = 2, numFeatures = 1

In [80]:
CURRENT_DIR 

'/home/jovyan/work/notebooks'

In [7]:
from datetime import date

In [9]:
# dd/mm/YY
today = date.today()
d1 = today.strftime("%d%m%Y")
print("d1 =", d1)

d1 = 22042020
